# Surface Water Estimation and Methodological Decisions

This notebook focuses on the estimation of surface water extent using satellite imagery.
Rather than presenting a single optimal method, the analysis emphasizes how methodological
choices influence results and their interpretation in a decision-making context.


## 1. Decision context

Surface water estimates derived from satellite data are frequently used to support
environmental monitoring, planning, and risk assessment.

However, these estimates depend on a series of methodological decisions, including the
choice of spectral indices, threshold values, and temporal aggregation strategies.


## 2. Surface water as a measurable variable

From a remote sensing perspective, surface water is not directly observed, but inferred
from spectral responses.

This inference introduces uncertainty and requires explicit assumptions that must be
acknowledged when results are used for decision-making.


## 3. Spectral indices for water detection

Several spectral indices have been proposed to enhance the detection of surface water.
In this project, the focus is placed on commonly used indices to ensure transparency and
reproducibility.

Typical indices include:
- NDWI (Normalized Difference Water Index)
- MNDWI (Modified Normalized Difference Water Index)


In [ ]:
# Placeholder for index computation
# Example (conceptual):
# ndwi = image.normalizedDifference(['GREEN', 'NIR'])

pass

## 4. Threshold selection as a decision

Transforming continuous index values into binary water / non-water classifications
requires the selection of threshold values.

Threshold selection is a decision that directly affects estimated water extent and
classification uncertainty.


## 5. Sensitivity to threshold variation

Small variations in threshold values may lead to substantial differences in the
estimated surface water area.

This sensitivity highlights the importance of evaluating multiple thresholds rather
than relying on a single fixed value.


## 6. Temporal aggregation choices

Surface water dynamics are inherently temporal. Decisions regarding temporal aggregation
(e.g., monthly vs. annual composites) influence the interpretation of variability and
extremes.


## 7. Uncertainty and interpretation

Uncertainty in surface water estimation arises from:

- Sensor characteristics
- Atmospheric effects
- Mixed pixels
- Methodological decisions

Recognizing these sources is essential for responsible use of results in
decision-making processes.


## 8. Role of this notebook in the workflow

This notebook introduces the methodological foundations for surface water estimation.
It prepares the data and assumptions required for subsequent temporal analysis.

The following notebook focuses on temporal trends and their implications for
decision-making.
